In [5]:
from typing import Dict, Optional
from chainlit.types import ThreadDict
from langchain.memory import ConversationBufferMemory
from langchain.schema.runnable.config import RunnableConfig 
# from chatbot_memory import setup_runnable
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain_google_vertexai import ChatVertexAI
import chainlit as cl

from rich import print as rprint

In [12]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True)


memory.chat_memory.add_user_message("Hello")

memory.chat_memory.add_ai_message("Test")

memory

ConversationBufferMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}), AIMessage(content='Test', additional_kwargs={}, response_metadata={})]), return_messages=True)

In [15]:

from langchain_core.runnables import RunnableMap, RunnablePassthrough, RunnableLambda

_inputs = RunnableMap(
        chat_history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"),
        question=itemgetter("question"),
    )

# Create a test input
test_input = {
    "question": "What's the circadian rhythm?"
}

In [18]:
# Run the _inputs with test input
output = _inputs.invoke(test_input)
output

{'chat_history': [HumanMessage(content='Hello', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Test', additional_kwargs={}, response_metadata={})],
 'question': "What's the circadian rhythm?"}

In [27]:
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, format_document
# Define prompt templates
RAG_TEMPLATE = """Given the following conversational history and user question, \
respond only with a search query to retrieve useful information from a medical \
knowledgebase. Do not narrate your thought process. If no relevant question is \
provided, respond only with "No valid query".

Chat History:
{chat_history}
Question: {question}
Search Query:"""

ANSWER_TEMPLATE = """You are a scientific expert in human wellness, health \
optimization, and athletic performance. Answer the user's question with relevant \
context from a medical knowledgebase if available. If the user's question is \
unrelated, politely reassert your role.

Context:
{context}

Question: {question}"""

# Instantiate templates
RAG_QUERY_PROMPT = PromptTemplate.from_template(RAG_TEMPLATE)
ANSWER_PROMPT = ChatPromptTemplate.from_template(ANSWER_TEMPLATE)
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template="{page_content}")


_rag_query = RunnableMap(
        rag_query = RAG_QUERY_PROMPT | ChatVertexAI(model="gemini-1.5-flash", temperature=0) | StrOutputParser(),
        question = itemgetter("question")
    )

rag_query_output = _rag_query.invoke(output)

In [28]:
rag_query_output

{'rag_query': 'circadian rhythm definition \n',
 'question': "What's the circadian rhythm?"}

In [29]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from vertexai.language_models import TextEmbeddingModel
from langchain_google_vertexai.embeddings import VertexAIEmbeddings

# Helper function to combine retrieved documents
def _combine_documents(
    docs, document_prompt=DEFAULT_DOCUMENT_PROMPT, document_separator="\n\n"
):
    """Combine documents into a single string."""
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)

# Setup embedding model and retriever
embedding_model = TextEmbeddingModel.from_pretrained("text-embedding-004")
client = QdrantClient(url="http://localhost:6333")
vectorstore = QdrantVectorStore(
    client=client,
    collection_name="uptodate",
    embedding=VertexAIEmbeddings(model_name="text-embedding-004")
)
retriever = vectorstore.as_retriever()

2024-10-29 17:36:38 - HTTP Request: GET http://localhost:6333/collections/uptodate "HTTP/1.1 200 OK"


In [30]:
_context = RunnableMap(
        context=itemgetter("rag_query") | retriever | _combine_documents,
        question=itemgetter("question")
    )

context_output = _context.invoke(rag_query_output)


2024-10-29 17:37:01 - HTTP Request: POST http://localhost:6333/collections/uptodate/points/query "HTTP/1.1 200 OK"


In [31]:
context_output

{'context': '●A medical problem, such as:\n\n\n\n•Hypothyroidism – This is the medical term for when a person does not make enough thyroid hormone. This hormone controls how your body uses and stores energy.\n\n\n\n•Depression – People with this condition feel sad or down a lot of the time. They often also have trouble working or doing everyday tasks.\n\n\n\n●Things that disturb your sleep, such as:\n\n\n\n•Sounds – For example, if you have a new baby, they might cry and wake you up at night.\n\n## Is there anything I can do on my own to feel better?\n\nThat depends on the cause of your daytime sleepiness. But you can try having good sleep habits. This is also called "sleep hygiene"  (table 1). You should:\n\n●Go to bed and get up at the same time every day.\n\n\n\n●Have drinks with caffeine in them only in the morning (including coffee and tea).\n\n\n\n●Avoid alcohol.\n\n\n\n●Avoid smoking, especially in the evening.\n\n\n\n●Lose weight, if you are overweight.\n\n## More on this topic